In [2]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
MODEL_SAVE_PATH = "Distil-BERT 모델이 저장된 위치를 입력해주세요." # 모델은 compass 폴더 바깥에 저장해주세요!! git에 올리면 복잡해집니다.
BASE_FOLDER = "compass 폴더의 위치를 지정해주세요"
target_data_path = os.path.join(BASE_FOLDER, "data/garments.csv")

In [4]:
# Use saved model - 로컬에 저장된 모델 불러오기
NUM_LABELS = 5
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH, num_labels=NUM_LABELS, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH, local_files_only=True)

Some layers from the model checkpoint at /content/drive/MyDrive/zb-2nd/distilbert-top5-sentitext were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/zb-2nd/distilbert-top5-sentitext and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferen

In [5]:
base_df = pd.read_csv(target_data_path).dropna()
base_df.head(2)

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspect,SentimentText,SentimentWord,SentimentPolarity
0,128481,아들에게 선물했는데 불편하고 활동하기 안좋다고 잘 안입고 다니네요,쇼핑몰,패션,남성의류,OO 남성 매** 데님 3종,100,37,8,20180626,-1.0,착용감,불편하고,1,-1
1,128481,아들에게 선물했는데 불편하고 활동하기 안좋다고 잘 안입고 다니네요,쇼핑몰,패션,남성의류,OO 남성 매** 데님 3종,100,37,8,20180626,-1.0,착용감,활동하기 안좋다고 잘 안입고 다니네요,5,-1


In [6]:
targets = ["디자인", "사이즈", "가격", "품질", "기능"]
target_df = base_df.loc[base_df["Aspect"].isin(targets), :]
target_df.head(2)

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspect,SentimentText,SentimentWord,SentimentPolarity
4,128484,이번에구매한데님은사이즈가잘맞네요 색상구성도괜찮고맘에든답니다 잘입겠습니다,쇼핑몰,패션,남성의류,OO 남성 매** 데님 3종,100,39,3,20180315,1.0,사이즈,사이즈가잘맞네요,1,1
15,128494,바지는 너무 편하고 좋은데 좀크게나온듯 그리고 허리고리 하나가 안달려서 밑단수선하면...,쇼핑몰,패션,남성의류,OO 남성 매** 데님 3종,60,118,24,20180317,0.0,사이즈,좀크게나온듯,1,-1


In [7]:
target_df[["RawText", "Aspect"]][:10]

,RawText,Aspect
4,이번에구매한데님은사이즈가잘맞네요 색상구성도괜찮고맘에든답니다 잘입겠습니다,사이즈
15,바지는 너무 편하고 좋은데 좀크게나온듯 그리고 허리고리 하나가 안달려서 밑단수선하면...,사이즈
19,신랑 바지가 없어서 사줬는데 엄청 입기 편하다고 좋아하네요 똑같은 라인으로 여성용 ...,가격
22,핏이 너무너무 좋아요 디자인도 색상도 너무 좋아요 가격대비 품질 짱이네요,디자인
24,핏이 너무너무 좋아요 디자인도 색상도 너무 좋아요 가격대비 품질 짱이네요,품질
25,190cm 93kg 체형에 36을 살까하다가 많은 분들의 후기로 한치수 내려 34로...,사이즈
27,190cm 93kg 체형에 36을 살까하다가 많은 분들의 후기로 한치수 내려 34로...,가격
29,디자인도예쁘고워싱도예쁘고 입으니 편하다고하네요. 50대신랑 핏이 젊어보여요.ㅎㅎ 잘...,디자인
35,이렇게 사이즈가 딱 맞다니 ㅋㅋㅋ 그것보다 착용감이 대기 편하고 신축성이 정말 좋음~,사이즈
38,별로에요~ 사이즈가 1개는 크네요. 균일하게 제작하는 세심함이 필요합니다. 3벌중 ...,사이즈


In [8]:
def test(x):
    labels = ["디자인", "사이즈", "가격", "품질", "기능"]
    aspects = x["Aspect"].to_list()
    result = []
    for label in labels:
        if label in aspects:
            result.append(1)
        else:
            result.append(0)
    return np.array(result)

onehot_df = target_df.groupby("RawText").apply(test).reset_index().rename(columns={ 0: "LabelList" })
onehot_df

,RawText,LabelList
0,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,"[1, 0, 1, 0, 0]"
1,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,"[0, 0, 0, 1, 0]"
2,그냥 착한가격이라 그냥 입기로 했지만 타사보다 사이즈가 좀크게 나왔어요,"[0, 1, 1, 0, 0]"
3,딸다리가 길아 기모바지를 사주어도 다~ 내가 입게되어 속상했는데 기장도 길고 밍크...,"[0, 0, 0, 0, 1]"
4,직립금으로 좀 할인해서 구매했는데요. 블랙이 있으면 좋았겠지만 없어서 다크레드로 ...,"[0, 0, 0, 1, 0]"
...,...,...
35627,힙라인위쪽이 배를감싸주게 올라온것이 너무 마음에들어요~~ 역시 편안함도 ㅇㅇ이네요,"[1, 0, 0, 0, 0]"
35628,"힙라인이 골반이 작은 저에게는 엉성하긴 하게 크지만, 소재감이 너무 좋고 얇아 착용...","[0, 1, 0, 0, 0]"
35629,힙부분은 사이즈가 여유롭지만 다리 부분은 타이트합니다. ..길이는..좀 짧은것 같아...,"[0, 1, 0, 0, 0]"
35630,힙있는55사이즈인데 27샀다가 엉덩이가 너무타이트해서28로 교환했어요 기모가있어서그...,"[0, 1, 1, 0, 1]"


In [10]:
def get_aspects(target_model, text_list):
    X_encoding = tokenizer(text_list, padding=True, truncation=True, max_length=42)
    X_formatted = tf.data.Dataset.from_tensor_slices(dict(X_encoding))
    return target_model.predict(X_formatted)

In [11]:
raw_texts = onehot_df.loc[:, "RawText"].to_list()
raw_texts[:5]

['   신발  편하고  예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조금은 덥긴하네요       ',
 '  착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기로 했어요         ',
 ' 그냥  착한가격이라 그냥 입기로 했지만  타사보다 사이즈가  좀크게 나왔어요',
 ' 딸다리가 길아 기모바지를 사주어도 다~ 내가 입게되어 속상했는데 기장도 길고 밍크라 겨울철 넘 따뜻하게 보내겠어요',
 ' 직립금으로 좀 할인해서 구매했는데요. 블랙이 있으면 좋았겠지만 없어서 다크레드로 구입했어요. 가방크기때문에 호불호 많던데 받아보니 보통체격의 여자분은 무난하게 착용할 수 있네요. 괜한 걱정한듯. 내구성, 제품의 질도 가격에 비해 좋아요.잘 산듯']

In [12]:
pred_aspect_list = get_aspects(model, raw_texts)
pred_aspect_list

35632/35632 [==============================] - 5513s 155ms/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 2.2611153 , -2.0802803 ,  2.0205135 , -2.4404478 , -1.590691  ],
       [-1.7596719 , -0.87059915, -0.91030425, -1.6247628 ,  4.2707705 ],
       [ 3.1217144 , -3.3057523 , -2.920321  ,  0.37703547, -0.09234041],
       ...,
       [-3.5942078 , -3.4409225 , -0.7088085 ,  5.6113825 , -0.7517926 ],
       [-3.0486743 , -3.4014707 , -1.5180156 ,  5.968751  , -1.325499  ],
       [-2.6584272 ,  1.6520447 ,  0.6552385 ,  0.5959132 , -0.8349405 ]],
      dtype=float32), hidden_states=None, attentions=None)

In [41]:
pred_aspect_logits = pd.Series(list(pred_aspect_list.logits))
pred_aspect_logits.head()

0    [2.2611153, -2.0802803, 2.0205135, -2.4404478,...
1    [-1.7596719, -0.87059915, -0.91030425, -1.6247...
2    [3.1217144, -3.3057523, -2.920321, 0.37703547,...
3    [-2.8631616, 4.830046, -1.258243, -1.2631577, ...
4    [4.275024, -2.9984868, -1.2868881, -2.4268546,...
dtype: object

In [42]:
onehot_df["PredLogits"] = pred_aspect_logits
onehot_df.head()

,RawText,LabelList,PredList,PredLogits
0,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,"[1, 0, 1, 0, 0]","[1, 0, 1, 0, 0]","[2.2611153, -2.0802803, 2.0205135, -2.4404478,..."
1,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,"[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[-1.7596719, -0.87059915, -0.91030425, -1.6247..."
2,그냥 착한가격이라 그냥 입기로 했지만 타사보다 사이즈가 좀크게 나왔어요,"[0, 1, 1, 0, 0]","[1, 0, 0, 1, 1]","[3.1217144, -3.3057523, -2.920321, 0.37703547,..."
3,딸다리가 길아 기모바지를 사주어도 다~ 내가 입게되어 속상했는데 기장도 길고 밍크...,"[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[-2.8631616, 4.830046, -1.258243, -1.2631577, ..."
4,직립금으로 좀 할인해서 구매했는데요. 블랙이 있으면 좋았겠지만 없어서 다크레드로 ...,"[0, 0, 0, 1, 0]","[1, 0, 0, 0, 1]","[4.275024, -2.9984868, -1.2868881, -2.4268546,..."


In [47]:
onehot_df.to_csv("onehot_garments_pred.csv", index=False)